In [1]:
from typing import List, Tuple

import pandas as pd
import torch
import numpy as np

from module import Word2IdConverter, sentence2words

In [2]:
train_df = pd.read_csv('data/train.txt', sep='\t')
train_df.shape

(10672, 8)

In [3]:
converter = Word2IdConverter('data/mapping.csv')
n_words = converter.get_n_words()

In [4]:
results = map(sentence2words, train_df.title)
results = map(converter.word2id, results)
X = list(map(lambda x: torch.Tensor(x).long(), results))
len(X)

10672

In [15]:
class CNN(torch.nn.Module):
    def __init__(self,
                 vocab_size: int,
                 input_size: int,
                 hidden_size: int,
                 output_size: int,
                 kernel_size: int):
        super(CNN, self).__init__()
        self._hidden_size = hidden_size
        self._kernel_size = kernel_size
        self.embedding = torch.nn.Embedding(vocab_size, input_size)
        self.conv = torch.nn.Conv2d(
            in_channels=1,
            out_channels=hidden_size,
            kernel_size=kernel_size,
            stride=1,
            padding=1,
            groups=1,
            bias=True
        )
        #self.linear = torch.nn.Linear(hidden_size*kernel_size*kernel_size, output_size)
        self.linear = torch.nn.Linear(hidden_size*6*100, output_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        print(f'1: {x.shape}')
        x = self.embedding(x)
        print(f'2: {x.shape}')
        x = x.unsqueeze(1)
        print(f'3: {x.shape}')
        x = self.conv(x)
        print(f'4: {x.shape}')
        x = torch.max_pool2d(x, kernel_size=(self._kernel_size, self._kernel_size))
        print(f'5: {x.shape}')
        #x = x.view(-1, self._hidden_size*self._kernel_size*self._kernel_size)
        x = x.view(-1, self._hidden_size*6*100)
        print(f'6: {x.shape}')
        x = self.linear(x)
        print(f'7: {x.shape}')
        return torch.log_softmax(x, dim=1)

In [16]:
dw = 300
dh = 50
n_class = 4
kernel_size = 3
cnn = CNN(vocab_size=n_words, input_size=dw, hidden_size=dh, output_size=n_class, kernel_size=kernel_size)

In [17]:
n_train_size = 100
X_train = X[:n_train_size]
X_pad = torch.nn.utils.rnn.pad_sequence(X_train, batch_first=True)

print(X_pad.shape)
y_pred = cnn(X_pad)
y_pred.shape

torch.Size([100, 18])
1: torch.Size([100, 18])
2: torch.Size([100, 18, 300])
3: torch.Size([100, 1, 18, 300])
4: torch.Size([100, 50, 18, 300])
5: torch.Size([100, 50, 6, 100])
6: torch.Size([100, 30000])
7: torch.Size([100, 4])


torch.Size([100, 4])

In [18]:
y_pred.argmax(dim=1)

tensor([2, 1, 1, 3, 3, 2, 3, 2, 3, 2, 1, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 2, 1, 3,
        2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2,
        2, 2, 2, 3, 3, 2, 1, 3, 3, 3, 3, 3, 2, 3, 1, 2, 2, 2, 3, 2, 3, 3, 3, 2,
        2, 2, 2, 3, 2, 2, 2, 3, 3, 3, 2, 2, 2, 2, 3, 2, 2, 2, 3, 3, 3, 2, 3, 3,
        3, 3, 2, 2])